In [1]:
using Gen
using PyCall
using ProgressBars
pushfirst!(PyVector(pyimport("sys")."path"), "");




In [17]:
SIR = pyimport("sir")
sns = pyimport("seaborn")
matplotlib = pyimport("matplotlib")
%matplotlib inline
plt = matplotlib.pyplot
model = SIR.SIR(1000)
S,I,R  = model.project(10)
I[end]

The analogue of IPython's `%matplotlib` in Julia is to use the [PyPlot package](https://github.com/stevengj/PyPlot.jl), which gives a Julia interface to Matplotlib including inline plots in IJulia notebooks.   (The equivalent of `numpy` is already loaded by default in Julia.)

Given PyPlot, the analogue of `%matplotlib inline` is `using PyPlot`, since PyPlot defaults to inline plots in IJulia.

To enable separate GUI windows in PyPlot, analogous to `%matplotlib`, do `using PyPlot; pygui(true)`.   To specify a particular gui backend, analogous to `%matplotlib gui`, you can either do `using PyPlot; pygui(:gui); using PyPlot; pygui(true)` (where `gui` is `wx`, `qt`, `tk`, or `gtk`), or you can do `ENV["MPLBACKEND"]=backend; using PyPlot; pygui(true)` (where `backend` is the name of a Matplotlib backend, like `tkagg`).

For more options, see the PyPlot documentation.


# Probabilistic Disease Model

In [3]:
projection_days = 90; 
N = 1300000000; #1.3 Billion population of India

sigma = 100

@gen function sample_SIR()
    
    #Sample date of introduction
    intro = @trace(poisson(10), :intro) #Expected introduction 10 days from now
    days = projection_days - intro
    
    log_r0 = @trace(normal(log(2.6), 0.5), :log_r0)
    r0 = exp(log_r0)
    
    inf_period = @trace(normal(4.5, 0.5), :inf_period)
    
    model = SIR.SIR(N, r0 = r0, inf_period = inf_period)
    #S,I,R = model.project(days)
    return model
end

@gen function final_infections(sigma)
    model = @trace(sample_SIR(), :model)
    #println(model)
    final_infected = model.final_infection()[1]*model.N
    obs = @trace(normal(final_infected, sigma), :obs)
    return final_infected
end

@gen function projection(days, sigma)
    model = @trace(sample_SIR(), :model)
    S,I,R = model.project(days)
    
    infected_population = I[end]
    obs = @trace(normal(infected_population, sigma), :obs)
    return infected_population
end

@gen function peak_case_load(days, sigma)
    model = @trace(sample_SIR(), :model)
    S,I,R = model.project(days)
    
    max_load = maximum(I)
    obs = @trace(normal(max_load, sigma), :obs)
    return max_load
end
    

tr = simulate(projection, (10,100))
println(get_choices(tr))
println(tr[:model => :log_r0])


┌ Warning: `lgamma(x::Real)` is deprecated, use `(logabsgamma(x))[1]` instead.
│   caller = traceat(::Gen.GFSimulateState, ::Gen.Poisson, ::Tuple{Int64}, ::Symbol) at poisson.jl:11
└ @ Gen /home/ajshah/.julia/packages/Gen/qL5ue/src/modeling_library/poisson.jl:11


│
├── :obs : 219.17518314213558
│
└── :model
    │
    ├── :log_r0 : 1.0478941997331028
    │
    ├── :inf_period : 5.43943326499934
    │
    └── :intro : 10

1.0478941997331028


# MH Sampling

**Gaussian drift proposal**


In [4]:
sigma_r0 = 0.001
sigma_inf = 1

#Block resampling for date of introduction

@gen function drift_proposal(tr, sigma_r0, sigma_inf)
    
    #perturb r0
    @trace(normal(tr[:model => :log_r0], sigma_r0), :model => :log_r0)
    
    #perturb infection period
    @trace(normal(tr[:model => :inf_period], sigma_inf), :model => :inf_period)
    
    #@trace(poisson(10), :model => :intro)
end

# tr = simulate(projection,(10, 100))
# println(get_choices(tr))
# tr = simulate(drift_proposal, (tr, sigma_r0, sigma_inf))
# println(get_choices(tr))

    

┌ Warning: `lgamma(x::Real)` is deprecated, use `(logabsgamma(x))[1]` instead.
│   caller = logpdf at poisson.jl:11 [inlined]
└ @ Core /home/ajshah/.julia/packages/Gen/qL5ue/src/modeling_library/poisson.jl:11
┌ Warning: `lgamma(x::Real)` is deprecated, use `(logabsgamma(x))[1]` instead.
│   caller = traceat(::Gen.GFUpdateState, ::Gen.Poisson, ::Tuple{Int64}, ::Symbol) at poisson.jl:11
└ @ Gen /home/ajshah/.julia/packages/Gen/qL5ue/src/modeling_library/poisson.jl:11


100.00%┣█████████████████████████████████████████████████████▉┫ 10000/10000 00:19<00:00, 538.72 it/s]
95.20%┣████████████████████████████████████████████████████▎  ┫ 47599/50000 01:26<00:04, 556.00 it/s]

Excessive output truncated after 524318 bytes.

95.32%┣████████████████████████████████████████████████████▍  

5

**Final Infections**

In [23]:
observed_final_infections = 100000000 #100 million final infections
nsamples = 50000
burn = 10000
samples = []
r0s = []
inf_periods = []

constraints = choicemap()
constraints[:obs] = observed_final_infections

#Draw and discard burning samples
tr,_ = generate(final_infections, (100,), constraints)
push!(samples, tr)
for i = tqdm(1:burn)    
    (tr, _) = mh(tr, drift_proposal, (sigma_r0, sigma_inf))
end

#Now collect the samples
for i = tqdm(1:nsamples)
    (tr, flag) = mh(tr, drift_proposal, (sigma_r0, sigma_inf))
    if flag
        push!(samples, tr)
    end
end

r0s_final = []
inf_periods_final = []
for i = 2:length(samples)
    push!(r0s_final, exp(samples[i][:model=>:log_r0]))
    push!(inf_periods_final, samples[i][:model=>:inf_period])
end


100.00%┣████████████████████████████████████████████████████▉┫ 10000/10000 00:03<00:00, 3601.38 it/s]
100.00%┣████████████████████████████████████████████████████▉┫ 50000/50000 00:14<00:00, 3529.89 it/s]


In [26]:
println(r0s_final)

Any[1.04056, 1.04056, 1.04056, 1.04056, 1.04056, 1.04055, 1.04056, 1.04055, 1.04055, 1.04056, 1.04056]


**Projections by Nth day**

In [28]:
observed_final_infections = 100000000 #100 million infections by day 100
nsamples = 50000
burn = 10000
samples = []
r0s = []
inf_periods = []

constraints = choicemap()
constraints[:obs] = observed_final_infections

#Draw and discard burning samples
tr,_ = generate(projection, (100,100), constraints)
push!(samples, tr)
for i = tqdm(1:burn)    
    (tr, _) = mh(tr, drift_proposal, (sigma_r0, sigma_inf))
end

#Now collect the samples
for i = tqdm(1:nsamples)
    (tr, flag) = mh(tr, drift_proposal, (sigma_r0, sigma_inf))
    if flag
        push!(samples, tr)
    end
end

r0s_projected = []
inf_periods_projected = []
for i = 2:length(samples)
    push!(r0s_projected, exp(samples[i][:model=>:log_r0]))
    push!(inf_periods_projected, samples[i][:model=>:inf_period])
end


println(r0s_projected)



100.00%┣█████████████████████████████████████████████████████▉┫ 10000/10000 00:20<00:00, 508.14 it/s]
87.62%┣████████████████████████████████████████████████▏      ┫ 43811/50000 01:26<00:12, 512.34 it/s]

Excessive output truncated after 524364 bytes.

In [29]:
println(r0s_projected)
println(inf_periods_projected)

Any[3.16925, 3.16974]
Any[0.233223, 0.159224]


**Peak Case Load**

In [31]:
observed_final_infections = 100000000 #100 million peak case load
nsamples = 50000
burn = 10000
samples = []

constraints = choicemap()
constraints[:obs] = observed_final_infections

#Draw and discard burning samples
tr,_ = generate(peak_case_load, (100,100), constraints)

for i = tqdm(1:burn)    
    (tr, _) = mh(tr, drift_proposal, (sigma_r0, sigma_inf))
end
push!(samples, tr)
#Now collect the samples
for i = tqdm(1:nsamples)
    (tr, flag) = mh(tr, drift_proposal, (sigma_r0, sigma_inf))
    if flag
        push!(samples, tr)
    end
end

r0s_projected = []
inf_periods_projected = []
for i = 2:length(samples)
    push!(r0s_projected, exp(samples[i][:model=>:log_r0]))
    push!(inf_periods_projected, samples[i][:model=>:inf_period])
end


println(r0s_projected)

100.00%┣█████████████████████████████████████████████████████▉┫ 10000/10000 00:15<00:00, 664.92 it/s]
100.00%┣█████████████████████████████████████████████████████▉┫ 50000/50000 01:09<00:00, 722.07 it/s]
Any[1.57642, 1.57679, 1.57659, 1.57645, 1.57643, 1.5772]


In [32]:
println(r0s_projected)
println(inf_periods_projected)

Any[1.57642, 1.57679, 1.57659, 1.57645, 1.57643, 1.5772]
Any[2.79053, 2.88949, 2.88513, 2.88073, 2.50221, 2.80899]


## Annealed Importance Sampling

BoundsError: BoundsError: attempt to access 0-element Array{Any,1} at index [1]